In [30]:
import os
import re

import unicodedata
import time
import pandas as pd
import numpy as np
from rapidfuzz import process, fuzz
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from transformers import logging as transformers_logging
from tabulate import tabulate
transformers_logging.set_verbosity_error()
import json

In [31]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words_to_keep = ["what", "when", "where", "which", "while", "who", "whom", "why", "with", "how", "before", "after","same"]
stop_words = set([s for s in stopwords.words('english') if s not in stop_words_to_keep])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sandr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
class NERParser:
    def __init__(self, model_name: str = "dslim/bert-base-NER", lowercase: bool = False):
        """
        Initialize the NER parser with a model and optionally configure the lowercase preprocessing.
        """
        self.model_name = model_name
        self.lowercase = lowercase
        self.device = self.get_device()
        
        # Load the tokenizer and model
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name, do_lower_case=self.lowercase)
        self.model = AutoModelForTokenClassification.from_pretrained(self.model_name)
        
        # Set up the NER pipeline
        self.nlp_pipeline = pipeline(
            "ner", 
            model=self.model, 
            tokenizer=self.tokenizer, 
            device=self.device, 
            aggregation_strategy="simple"
        )

    def get_device(self):
        """
        Determines whether to use MPS, CUDA, or CPU depending on the available hardware.
        """
        if torch.backends.mps.is_available():
            print("MPS device found, using MPS backend.\n")
            return torch.device("mps")
        elif torch.cuda.is_available():
            print(f"CUDA device found, using CUDA backend. Device: {torch.cuda.get_device_name(0)}\n")
            return torch.device("cuda")
        else:
            print("Neither MPS nor CUDA found, using CPU.\n")
            return torch.device("cpu")

    
    def parse_ner_results(self, ner_results: list):
        """
        Parse the NER results and extract entities related to 'PER' (persons) and 'MISC' (potential movie titles).
        """
        per_entities, misc_entities = [], []
        
        for entity in ner_results:
            # Extraction of all Persons
            if entity['entity_group'] == 'PER':
                per_entities.append(entity['word'])
            # Extraction of all Misc that could indicate movies
            elif entity['entity_group'] == 'MISC':
                misc_entities.append(entity['word'])
        
        return per_entities, misc_entities

    
    def process_query(self, query: str):
        """
        Processes a text query, runs NER, and returns the extracted actors and movie names.
        """
        # Optionally lowercase the input if configured
        if self.lowercase:
            query = query.lower()
        
        # Run the NER pipeline
        ner_results = self.nlp_pipeline(query)

        # Parse the results to extract actors and movies
        per_entities, misc_entities = self.parse_ner_results(ner_results)
        
        return per_entities, misc_entities

ner_parser = NERParser(lowercase=False)

Neither MPS nor CUDA found, using CPU.



In [51]:
class DataBase:
    """Handles the extraction of context data for given people and movies from a database, with fuzzy matching for names."""

    def __init__(self):
        self.db = pd.read_pickle(os.path.join(os.getcwd(), r"exports/extended_graph_triples.pkl"))
        self.reverse_index = pd.read_pickle(os.path.join(os.getcwd(), r"exports/reverse_index.pkl"))
        self.reverse_index['normalized_subject_label'] = self.reverse_index['subject_label'].apply(self.normalize_string)
        
        with open(r"exports/entity_db.json", encoding="utf-8") as f: 
            self.entities = json.load(f)
            self.entity_list = list(subject.lower() for subject, types in self.entities.values())
        
        # otherwise exact matching will fail
        self.db['subject_id'] = self.db['subject_id'].astype(str).str.strip()


    def normalize_string(self, s):
        """Cleans the input entity to a uniform naming convention, by removing non ascii characters, encoding it to utf, setting it to lowercase, and removing redundant spaces"""
        s = s.lower()
        s = unicodedata.normalize('NFKD', s)
        s = s.encode('ascii', 'ignore').decode('utf-8')
        s = re.sub(r'[^\w\s]', '', s)
        s = ' '.join(s.split())
        return s


    def get_context(self, people: list, movies: list):
        """Fetch context data for the given people and movies from the database.
        First tries matching with the entire people or movie string. If that fails,
        it processes each person or movie individually.

        Args:
            people (list): List of people names to search.
            movies (list): List of movie titles to search.

        Returns:
            pd.DataFrame: DataFrame containing context for the given people and movies.
        """
        combined_people = " ".join(people) if people else ""
        combined_movies = " ".join(movies) if movies else ""
        
        context_person = pd.DataFrame()
        context_movie = pd.DataFrame()

        if combined_people:
            context_person = self._fetch_relevant(combined_people)
        
        if combined_movies:
            context_movie = self._fetch_relevant(combined_movies)
        
        if context_person.empty and people:
            person_results = [self._fetch_relevant(p) for p in people]
            if any(not res.empty for res in person_results):
                context_person = pd.concat(person_results, axis=1)

        if context_movie.empty and movies:
            movie_results = [self._fetch_relevant(m) for m in movies]
            if any(not res.empty for res in movie_results):
                context_movie = pd.concat(movie_results, axis=1)

        if not context_person.empty and not context_movie.empty:
            return pd.concat([context_person, context_movie], axis=1)
        
        elif not context_person.empty:
            return context_person
        
        elif not context_movie.empty:
            return context_movie
        
        else:
            return pd.DataFrame()
        
    def fetch_by_id_lst(self, subject_ids: list):
        relevant = self.db[self.db.subject_id.isin(subject_ids)].dropna(axis=1)
        
        if relevant.empty:
            print(f"No context data found for any in '{subject_ids}'.")
            return pd.DataFrame()
        
        pivot_df = relevant.pivot_table(
            index='subject_id',
            columns='predicate_label',
            values='object_label',
            aggfunc=lambda x: ' | '.join(x.astype(str))
        )
    
        pivot_df.reset_index(inplace=True)
    
        return pivot_df

    def _fetch_relevant(self, name: str):
        """Fetch relevant information for a given name (either combined or individual) from the database.
        Uses fuzzy matching if no exact match is found.

        Args:
            name (str): The name of the person or movie to search for.

        Returns:
            pd.DataFrame: Relevant data for the name, or an empty DataFrame if no match is found.
        """
        
        name = self.normalize_string(name)
        
        start = time.time_ns()
        matches = process.extractOne(name, self.reverse_index.normalized_subject_label, scorer=fuzz.ratio)
        print(f"Time taken for fuzzy matching: {(time.time_ns() - start) / 1e6} ms")
        
        if not matches or not matches[1] > 90:
            return pd.DataFrame()
        
        match_label = matches[0]
        print(f"Matched '{name}' to '{match_label}' with a score of {matches[1]}.")
        
        start = time.time_ns()
        lst_of_ids = self.reverse_index.get(self.reverse_index.normalized_subject_label == match_label, []).subject_id.to_list()
        print(f"Time taken for reverse index lookup: {(time.time_ns() - start) / 1e6} ms")
        
        if len(lst_of_ids) == 0:
            return pd.DataFrame()
        
        start = time.time_ns()
        relevant = self.db[self.db.subject_id.isin(lst_of_ids)].dropna(axis=1)
        print(f"Time taken for fetching data: {(time.time_ns() - start) / 1e6} ms")

        if relevant.empty:
            print(f"No context data found for '{match_label}'. Performing partial match search.")
            start = time.time_ns()
            relevant = self.db[self.db.subject_label.str.lower().str.startswith(name.lower())].dropna(axis=1)
            print(f"Time taken for partial match search: {(time.time_ns() - start) / 1e6} ms")
            
        if relevant.empty:
            print(f"No context data found for '{match_label}', even with partial match search.")
            return pd.DataFrame()
        
        start = time.time_ns()
        pivot_df = relevant.pivot_table(
            index='subject_id',
            columns='predicate_label',
            values='object_label',
            aggfunc=lambda x: ' | '.join(x.astype(str))
        )
    
        pivot_df.reset_index(inplace=True)
        print(f"Time taken for pivot table creation: {(time.time_ns() - start) / 1e6} ms")
    
        return pivot_df

In [52]:
class QueryEmbedder:
    
    def __init__(self):
        self._glove_embeddings = self._load_glove_embeddings("exports/glove.6B/glove.6B.300d.txt")
        
    def _load_glove_embeddings(self, file_path):
        embeddings = {}
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                values = line.split()
                word = values[0]
                vector = np.asarray(values[1:], dtype='float32')
                embeddings[word] = vector
        return embeddings
    
    def embed_phrase(self, phrase):
        words = phrase.split()
        word_vectors = [self._glove_embeddings[word.lower()] for word in words if word.lower() in self._glove_embeddings]
        
        if len(word_vectors) == 0:
            return np.zeros(300)
        
        return np.mean(word_vectors, axis=0)
    

In [53]:
class LLM():
    
    def __init__(self):
        self.qa_model = pipeline("question-answering", model="distilbert-base-cased-distilled-squad", top_k=1)
    
    def _query(self, query, context_df):
        
        context = ""
        for index, row in context_df.iterrows():
            row_context = " ".join([f"{col}: {row[col]}" for col in context_df[top_columns].columns])
            context += row_context + " "
        
        output = self.qa_model(question=query, context=context)
        
        answer_str = str()
        if isinstance(output, list):
            answer_str = ", ".join([result['answer'] for result in output])
            
        elif isinstance(output, dict):
            answer_str = output['answer']
        
        if not answer_str:
            answer_str = "No answer found."
        
        return answer_str

In [54]:
def filter_query(query, person, movies):
        
    person_list = [word.lower() for p in person for word in p.split(" ")] if len(person) else []
    movie_list = [word.lower() for m in movies for word in m.split(" ")] if len(movies) else []
        
    if not len(query):
        return []
    
    relevant = []
    for word in query.replace(". ", " ").lower().split(" "):
        cleaned_word = re.sub(r'[^A-Za-z]', '', word)
        if cleaned_word in stop_words or cleaned_word in person_list or cleaned_word in movie_list or cleaned_word == "":
            continue
        
        relevant.append(cleaned_word)
        
    return relevant

In [55]:
def cosine_sim(vec1, vec2):
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    
    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0.0 
    
    return np.dot(vec1, vec2) / (norm_vec1 * norm_vec2)

def find_closest_columns(query_embeddings, column_embeddings, top_n=5):
    column_similarities = {}

    for col, col_vec in column_embeddings.items():
        similarities = [cosine_sim(col_vec, q_vec) for q_vec in query_embeddings if np.linalg.norm(q_vec) > 0]
        column_similarities[col] = np.mean(similarities) if similarities else -1

    sorted_columns = sorted(column_similarities.items(), key=lambda item: item[1], reverse=True)
    
    return [col for col, sim in sorted_columns[:top_n]]

In [56]:
db = DataBase()

qe = QueryEmbedder()

llm = LLM()

In [57]:
# query = "Who directed The Bridge on the River Kwai?"
query = "Who is the director of Good Will Hunting?"

normalized_query = query.lower()

matches = process.extract(normalized_query, db.entity_list, scorer=fuzz.partial_ratio, limit=100)

adjusted_matches = []
for match in matches:
    name = match[0]
    score = match[1]
    matched_id = next(key for key, value in db.entities.items() if value[0] == name)
        
    length_ratio = len(name) / len(normalized_query)
    adjusted_score = score * length_ratio
    adjusted_matches.append((matched_id, name, adjusted_score))

threshold = 30

matched_names = [(name, score) for id, name, score in adjusted_matches if score >= threshold]

matched_entities = [id for id, name, score in adjusted_matches if score >= threshold]

print(tabulate(matched_names, headers=["Name", "Score"], tablefmt="grid"))

+-------------------+---------+
| Name              |   Score |
+===================+=========+
| good will hunting | 41.4634 |
+-------------------+---------+
| who gets the dog  | 30.662  |
+-------------------+---------+


In [58]:
context = db.fetch_by_id_lst(matched_entities)
context.head()

predicate_label,subject_id,aspect ratio,assessment,award received,box office,cast member,color,country of origin,ddis:rating,ddis:tag,...,nmhh film rating,node description,node label,nominated for,notable work,original language of film or tv show,production company,publication date,rars rating,screenwriter
0,http://www.wikidata.org/entity/Q193835,widescreen,"reverse bechdel test, bechdel test","academy award for best supporting actor, acade...",225900000,"harmony korine, stellan skarsgard, ben affleck...",color,united states of america,8.3,"inspiring, feel-good, humor, flashback, psycho...",...,category iv,1997 film by Gus Van Sant,Good Will Hunting,"academy award for best film editing, academy a...",minnie driver,english,"miramax, a band apart",1997-12-05,NaN,"matt damon, ben affleck"
1,http://www.wikidata.org/entity/Q27958314,NaN,NaN,NaN,NaN,NaN,NaN,united states of america,NaN,NaN,...,NaN,2016 film,Who Gets the Dog?,NaN,NaN,english,epic pictures group,2016-09-13,12,NaN


In [59]:
person, movies = ner_parser.process_query(query)

context = db.get_context(person,movies)
column_embeddings = {col: qe.embed_phrase(col) for col in context.columns}

filtered_query = filter_query(query, person, movies)
query_embeddings = [qe.embed_phrase(word) for word in filtered_query]

context.head()


Time taken for fuzzy matching: 0.0 ms
Matched 'good' to 'good' with a score of 100.0.
Time taken for reverse index lookup: 14.6609 ms
Time taken for fetching data: 15.865 ms
No context data found for 'good'. Performing partial match search.
Time taken for partial match search: 490.0803 ms
Time taken for pivot table creation: 112.915 ms


predicate_label,subject_id,after a work by,art director,aspect ratio,assessment,author,award received,based on,bbfc rating,box office,...,production designer,publication date,rcq classification,rtc film rating,screenwriter,set during recurring event,set in period,significant event,subclass of,takes place in fictional universe
0,http://www.wikidata.org/entity/Q10797504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,binary opposition,NaN
1,http://www.wikidata.org/entity/Q11972368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,http://www.wikidata.org/entity/Q12124618,NaN,NaN,NaN,bechdel test,NaN,NaN,NaN,NaN,NaN,...,NaN,2012-05-31,NaN,NaN,glenn patterson,NaN,NaN,NaN,NaN,NaN
3,http://www.wikidata.org/entity/Q14833945,NaN,NaN,NaN,bechdel test,NaN,NaN,NaN,NaN,NaN,...,NaN,2014-01-01,NaN,b15,"kelly masterson, marcus sakey",NaN,NaN,NaN,NaN,NaN
4,http://www.wikidata.org/entity/Q1537484,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2001-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
top_columns = find_closest_columns(query_embeddings, column_embeddings, top_n=10)
filtered_context_df = context[top_columns]

In [61]:
llm._query(query, filtered_context_df)

'jason priestley'

In [44]:
person

[]